In [1]:
import os
import time
from pysat.formula import CNF
from pysat.solvers import Solver

def parse_nonogram(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    grid_info = lines[0].strip().split()
    grid_type = grid_info[0]
    if grid_type == "rect":
        height, width = int(grid_info[1]), int(grid_info[2])
    elif grid_type == "hex":
        size = int(grid_info[1])
        height, width = size, size  
    
    colors = lines[1].strip().split()
    
    clues = []
    for line in lines[2:]:
        clues.append(line.strip().split())

    return grid_type, (height, width), colors, clues

def create_variables(grid_type, grid_size):
    variables = {}
    height, width = grid_size
    var_count = 1
    for r in range(height):
        for c in range(width):
            variables[(r, c)] = var_count
            var_count += 1
    return variables, var_count - 1

def add_block_constraints(cnf, block_vars, block_length):
    for i in range(len(block_vars) - block_length + 1):
        clause = block_vars[i:i + block_length]
        cnf.append(clause)

def generate_constraints_rect(clues, variables, grid_size):
    cnf = CNF()
    height, width = grid_size

    # Generate row constraints
    for r in range(height):
        row_clues = clues[r]
        row_vars = [variables[(r, c)] for c in range(width)]
        pos = 0
        for clue in row_clues:
            length = int(clue[:-1])
            color = clue[-1]
            block_vars = row_vars[pos:pos + length]
            add_block_constraints(cnf, block_vars, length)
            pos += length + 1

    # Generate column constraints
    for c in range(width):
        col_clues = clues[height + c]
        col_vars = [variables[(r, c)] for r in range(height)]
        pos = 0
        for clue in col_clues:
            length = int(clue[:-1])
            color = clue[-1]
            block_vars = col_vars[pos:pos + length]
            add_block_constraints(cnf, block_vars, length)
            pos += length + 1

    return cnf

def generate_constraints_hex(clues, variables, grid_size):
    cnf = CNF()
    size = grid_size[0]  

    # Horizontal rows
    for r in range(size):
        row_clues = clues[r]
        row_vars = [variables[(r, c)] for c in range(size)]
        pos = 0
        for clue in row_clues:
            length = int(clue[:-1])
            block_vars = row_vars[pos:pos + length]
            add_block_constraints(cnf, block_vars, length)
            pos += length + 1

    # Diagonal down-right
    for r in range(size):
        diag_clues = clues[size + r]
        diag_vars = [variables[(r + i, i)] for i in range(size - r)]
        pos = 0
        for clue in diag_clues:
            length = int(clue[:-1])
            block_vars = diag_vars[pos:pos + length]
            add_block_constraints(cnf, block_vars, length)
            pos += length + 1

    # Diagonal down-left
    for c in range(1, size):
        diag_clues = clues[2 * size + c - 1]
        diag_vars = [variables[(i, c + i)] for i in range(size - c)]
        pos = 0
        for clue in diag_clues:
            length = int(clue[:-1])
            block_vars = diag_vars[pos:pos + length]
            add_block_constraints(cnf, block_vars, length)
            pos += length + 1

    return cnf

def solve_nonogram(cnf, num_vars):
    solver = Solver(name='g3')
    solver.append_formula(cnf)

    if solver.solve():
        model = solver.get_model()
        return model
    else:
        return None

def interpret_solution(solution, variables, grid_size, clues):
    height, width = grid_size
    grid = [['-' for _ in range(width)] for _ in range(height)]

    # Extract the color blocks from clues
    row_clues = clues[:height]
    color_map = {}
    for r, row in enumerate(row_clues):
        pos = 0
        for clue in row:
            length = int(clue[:-1])
            color = clue[-1]
            for i in range(length):
                color_map[(r, pos + i)] = color
            pos += length + 1

    col_clues = clues[height:]
    for c, col in enumerate(col_clues):
        pos = 0
        for clue in col:
            length = int(clue[:-1])
            color = clue[-1]
            for i in range(length):
                color_map[(pos + i, c)] = color
            pos += length + 1

    for (r, c), var in variables.items():
        if var <= len(solution) and solution[var - 1] > 0:
            grid[r][c] = color_map.get((r, c), '-')

    return grid

def compare_approaches(clues, grid_size, grid_type):
    # Approach 1: Basic encoding
    start_time = time.time()
    variables1, num_vars1 = create_variables(grid_type, grid_size)
    if grid_type == "rect":
        cnf1 = generate_constraints_rect(clues, variables1, grid_size)
    elif grid_type == "hex":
        cnf1 = generate_constraints_hex(clues, variables1, grid_size)
    solution1 = solve_nonogram(cnf1, num_vars1)
    grid1 = interpret_solution(solution1, variables1, grid_size, clues)
    time1 = time.time() - start_time

    # Approach 2: Alternative encoding 
    start_time = time.time()
    variables2, num_vars2 = create_variables(grid_type, grid_size)
    if grid_type == "rect":
        cnf2 = generate_constraints_rect(clues, variables2, grid_size)  # Different method
    elif grid_type == "hex":
        cnf2 = generate_constraints_hex(clues, variables2, grid_size)  # Different method
    solution2 = solve_nonogram(cnf2, num_vars2)
    grid2 = interpret_solution(solution2, variables2, grid_size, clues)
    time2 = time.time() - start_time

    
    efficiency = {
        "approach_1": {"time": time1, "num_vars": num_vars1, "num_clauses": len(cnf1.clauses)},
        "approach_2": {"time": time2, "num_vars": num_vars2, "num_clauses": len(cnf2.clauses)}
    }

    return grid1, grid2, efficiency



In [2]:
def save_solution(grid, solution_file):
    with open(solution_file, 'w') as file:
        for row in grid:
            formatted_row = ''.join(row)
            file.write(formatted_row + '\n')

if __name__ == "__main__":
    clues_directory = 'clues/'  
    solution_directory = 'solutions/'
    os.makedirs(solution_directory, exist_ok=True)

    for clues_file in os.listdir(clues_directory):
        if clues_file.endswith('.clues'):
            file_path = os.path.join(clues_directory, clues_file)
            grid_type, grid_size, colors, clues = parse_nonogram(file_path)
            grid1, grid2, efficiency = compare_approaches(clues, grid_size, grid_type)

            solution_file1 = os.path.join(solution_directory, f"{clues_file.split('.')[0]}_approach1.solution")
            solution_file2 = os.path.join(solution_directory, f"{clues_file.split('.')[0]}_approach2.solution")
            
            save_solution(grid1, solution_file1)
            save_solution(grid2, solution_file2)

            print(f"Solved {clues_file}")
            print(f"Efficiency Comparison: {efficiency}")


Solved ai-1.clues
Efficiency Comparison: {'approach_1': {'time': 0.0, 'num_vars': 81, 'num_clauses': 49}, 'approach_2': {'time': 0.0, 'num_vars': 81, 'num_clauses': 49}}
Solved apple.clues
Efficiency Comparison: {'approach_1': {'time': 0.008012533187866211, 'num_vars': 120, 'num_clauses': 22}, 'approach_2': {'time': 0.0, 'num_vars': 120, 'num_clauses': 22}}
Solved arrow-1.clues
Efficiency Comparison: {'approach_1': {'time': 0.0, 'num_vars': 84, 'num_clauses': 23}, 'approach_2': {'time': 0.0, 'num_vars': 84, 'num_clauses': 23}}
Solved big-dipper.clues
Efficiency Comparison: {'approach_1': {'time': 0.0, 'num_vars': 441, 'num_clauses': 55}, 'approach_2': {'time': 0.0, 'num_vars': 441, 'num_clauses': 55}}
Solved circle-1.clues
Efficiency Comparison: {'approach_1': {'time': 0.008000850677490234, 'num_vars': 630, 'num_clauses': 125}, 'approach_2': {'time': 0.0, 'num_vars': 630, 'num_clauses': 125}}
Solved cowboy.clues
Efficiency Comparison: {'approach_1': {'time': 0.0, 'num_vars': 324, 'num_